## Composition and Encapsuation

As mentioned in the previous section, in my view, the foundations of object orientation are data composition and encapsulation, which most languages support to varying degrees. In classical OO, classes provide both. In Julia, composition is provided by structs, where encapsulation is provided by function methods (i.e. multiple dispatch).

### Structs

Composite data types in Julia are defined as `struct`s, very much as in C. Like C structs, they contain data fields and no methods. Unlike C, the definition of a struct always creates a new named data type (or type constructor, but more on that later). Here's an example of defining a struct and initializing an instance in Julia:

In [1]:
# declare your struct
struct Point
    x::Float64
    y::Float64
end

mypoint = Point(5, 7)

Point(5.0, 7.0)

It's probably obvious, but this defines a `Point` type that contains the fields `x` and `y`, both of which are instances of Float64. `mypoint` is an instance of Point where `x` = 5.0 and `y` = 7.0.

**Note on type declarations in fields:**

> Fields of structs in Julia don't need to have type declarations. You could write this instead:
>
> ```julia
struct Point
    x
    y
end
>```
>
> However, type declarations on stucts (*not* on functions) are important for how Julia's optimizing compiler works (with type inference), so we're always going to use them in this tutorial. There will be more about creating inferable structs in the section on polymorphism.

Next, let's look at attribute access:

In [2]:
mypoint.x

5.0

So, that's easy and similar to other languages. You just use the object followed by a period and the name of the field. What if we want to change an attribute?

In [3]:
mypoint.x = 3.0

ErrorException: setfield! immutable struct of type Point cannot be changed

Oh dear, an error! Actually, this is a good thing. Structs in Julia are immutable by default. This is useful for a point, because a point only ever is what it is. If one of the coordinates changes, it's a different point. But let's say we want to make a starship (and nevermind that it's location would also have a z axis). Its location could change. For that, you need a mutable struct:

In [4]:
mutable struct Starship
    name::String
    location::Point
end

ship = Starship("U.S.S. Enterprise", Point(5, 7))

Starship("U.S.S. Enterprise", Point(5.0, 7.0))

We can move our ship by changing the location:

In [5]:
ship.location = Point(6, 8)

Point(6.0, 8.0)

In general, you should prefer to use immutable structs because they are easier to reason about (especially in concurrent contexts).

Now, having to use the `Point` constructor when we initialize our starship is kind of annoying. We can declare alternative constructors:

In [6]:
Starship(name, x, y) = Starship(name, Point(x, y))

othership = Starship("U.S.S. Defiant", 10, 2)

Starship("U.S.S. Defiant", Point(10.0, 2.0))

Julia also provides internal constructors, which *must* be used in the creation of new structs, using `new`. They are only needed in a few cases, which will be covered later, but here is a quick example:

In [7]:
mutable struct FancyStarship
    name::String
    location::Point
    FancyStarship(name, x, y) = new(name, Point(x, y))
end

fancy = FancyStarship("U.S.S. Discovery", 14, 32)

FancyStarship("U.S.S. Discovery", Point(14.0, 32.0))

However:

In [8]:
FancyStarship("U.S.S. Ticonderoga", Point(14, 32))

MethodError: MethodError: no method matching FancyStarship(::String, ::Point)
Closest candidates are:
  FancyStarship(::Any, ::Any, !Matched::Any) at In[7]:4

As you see, adding internal constructors means the basic constructor is no longer available to the outside. I typically only use internal constructors if using the basic constructor directly would somehow violate the purpose of the object.

### Methods

Generally, you want to provide a simple interface for your types so the user doesn't have to care about how it's implemented, just how to use it. Before, we moved our starship by changing its location directly, but our users shouldn't have to care about the internal data layout of our struct, so we can just give them a move function.

In [9]:
# I'm not a math person, and I nearly broke my head trying
# to find the right formula.
function move!(starship, heading, distance)
    old = starship.location
    x = distance * cosd(heading)
    y = distance * sind(heading)
    starship.location = Point(old.x + x, old.y + y)
end

ship = Starship("Foo", 3, 4)
move!(ship, 45, 1.5)
println(ship)

Starship("Foo", Point(4.060660171779821, 5.060660171779821))


And so the good ship Foo sails through the stars. The function name `move!` has an exclamation point at the end not because of my enthusiasm for trigonometry, but to indicate that it has *side effects*; i.e. it changes an object in place. This is not a rule in Julia, but it's a convention used in the standard library and elsewhere, and you should follow so it's easier to identify such functions.

Of course, `move!` is a very general function name, so you might want to add a type constraint to make sure you only get starships: `function move!(ship::Starship, heading, distance)`

**Note on data hiding:**

> Julia does not enforce data hiding, so anyone can access your fields directly. The convention is like Python: If a field's name starts with an underscore, you should consider it an implementation detail, and it is subject to change and produce strange effects if you mess with it. Use this convention in your own code.
>
> Immutable structs (the default) keep you from changing data that should be considered private, but it is still possible to act on its value which is also bad since the implementation could change.
> As a user of libraries, unless you're writing code that only has to run once, I would suggest not touching anyone else's data fields unless you're directed to do so in the library's documentation--whether or not they start with an underscore.
>
> You can make it more difficult to access fields on your structs by overloading `setproperty!` and `getproperty` methods for your type, but it's a bit overkill, and will require using `setfield!` and `getfield` in your own methods. Of course, others can use the same technique to access you're fields. It's basically the equivalent of a "No Trespassing" sign.

Anyway, you can add additional methods to any function--yes, the methods belong to the functions in Julia. Let's make some rectangles. (I have no idea why all my examples involve geometry, since I don't know anything about it).

In [10]:
struct Rectangle
    width::Float64
    height::Float64
end
width(r::Rectangle) = r.width
height(r::Rectangle) = r.height

struct Square
    length::Float64
end
width(s::Square) = s.length
height(s::Square) = s.length

area(shape) = width(shape) * height(shape)

r = Rectangle(3, 4)
s = Square(3)
@show area(r)
@show area(s);

area(r) = 12.0
area(s) = 9.0


So, if we wanted to to save some space, we could represent a square using the length of only one side, and we define `width` and `height` methods for it that return the same attribute, where as the implementation of width and height for rectangles needs to return distinct values.

When we define the `area` function, we don't need to care whether it's for a square or a rectangle. The Julia compiler selects the correct method for the type when it's needed. It doesn't matter what the type is, so long as it provides the correct interface. We are beginning to leak into the topic of polymorphism, so let's [head over that way](polymorphism.ipynb).